# VPC와 서브넷

## VPC
- Virtual Private Cloud는 사용자의 AWS 계정 전용 가상 네트워크입니다. VPC는 AWS 클라우드에서 다른 가상 네트워크와 논리적으로 분리되어 있습니다. Amazon EC2 인스턴스와 같은 AWS 리소스를 VPC에서 실행할 수 있습니다. IP 주소 범위와 VPC 범위를 설정하고 서브넷을 추가하고 보안 그룹을 연결한 다음 라우팅 테이블을 구성합니다
- 가상으로 존재하는 데이터센터
- 원하는 대로 사설망을 구축 가능
    - 부여된 IP 대역을 분할하여 사용 가능
- 리전 단위
- VPC의 사용 사례
    - EC2, RDS, Lambda 등의 AWS의 컴퓨팅 서비스 실행
    - 다양한 서브넷 구성
    - 보안 설정(IP Block, 인터넷에 노출되지 않는 EC2등 구성)

### VPC의 구성요소
- 서브넷
- 인터넷 게이트웨이
- NACL / 보안그룹
- 라우트 테이블
- NAT Instance / NAT Gateway
- Bastion Host
- VPC Endpoint

#### 서브넷
- VPC의 하위 단위로 VPC에 할당된 IP를 더 작은 단위로 분할한 개념
- 하나의 서브넷은 하나의 가용영역안에 위치
- CIDR Block range로 IP주소 지정
    - IPv4, IPv6
- IPv4의 경우 최소 /28 ~ 최대 /16
    - 즉 호스트 비트가 32-28=4, 2^4=16-5=11. 최소 11개 IP보유
- IPv6의 경우 /44 ~ /64

##### AWS 서브넷의 IP 개수 (왜 5개 빼야하나)
- 예 : 10.0.0.0/24 라면
    - 10.0.0.0 : 네트워크 어드레스
    - 10.0.0.1 : VPC Router
    - 10.0.0.2 : DNS Server
    - 10.0.0.3 : 미래에 사용하기 위해 비워 둠
    - 10.0.0.255 : 네트워크 브로드캐스트 어드레스(단, 브로드캐스트는 지원하지 않음)
    - 즉 총 사용 가능한 IP는 2^8-5=251개

#### VPC Router
- VPC에 있는 가상의 라우터. 서브넷에서 오고가는 트래픽을 라우팅
    - 즉 모든 서브넷의 트래픽은 VPC 라우터를 거쳐서 목적지에 도달
- VPC 생성 시 자동으로 생성되며 별도로 관리할 필요가 없음
    - 별도의 설정은 불가능하며, Route Table만 관리 가능

#### 라우트 테이블
- VPC 라우터에서 트래픽이 어디로 가야 할지 알려주는 이정표
- VPC 생성 시 기본으로 하나 제공
- 구성 요소
    - Destination : 트래픽이 가고자 하는 주소
    - Target : 트래픽을 실제로 보내줄 대상
        - 논리적 리소스의 아이디로 표현 (예 : Internet Gateway의 경우 IGW-xxxx)

#### 서브넷의 종류
- 퍼블릭 서브넷 : 외부에서 인터넷을 통해 연결할 수 있는 서브넷
    - 인터넷 게이트웨이를 통해 외부의 인터넷과 연결되어 있음
    - 안에 위치한 인스턴스에 퍼블릭 IP 부여 가능
    - 웹서버, 어플리케이션 서버 등 유저에게 노출되어야 하는 인프라
- 프라이빗 서브넷 : 외부에서 인터넷을 통해 연결할 수 없는 서브넷
    - 외부 인터넷으로 경로가 없음
    - 퍼블릭 IP 부여해봤자 의미가 없음
    - 데이터베이스, 로직 서버 등 외부에 노출될 필요가 없는 인프라

#### 인터넷 게이트웨이
- VPC가 외부의 인터넷과 통신할 수 있도록 경로를 만들어주는 리소스
- 기본적으로 확장성과 고가용성이 확보되어 있음
- IPv4, IPv6 지원
    - IPv4의 경우 NAT 역할도 수행
- Route Table에서 경로 설정 후에 접근 가능
- 무료

#### 기본 VPC와 커스텀 VPC
- 기본 VPC
    - AWS 계정 생성 시 자동으로 생성
    - 기본적으로 각 AZ마다 서브넷을 생성
        - 모든 서브넷이 인터넷 접근이 가능함(퍼블릭)
    - 다양한 AWS 서비스가 기본 VPC를 이용하기 때문에 삭제 시 여러 서비스 사용에 제약
- 커스텀 VPC
    - 직접 생성
    - 기본적으로 인터넷에 연결되어 있지 않음
        - 인터넷 게이트웨이와 라우팅 설정 없이 퍼블릭 서브넷 생성 불가능
        - 즉 별도의 조치 없이 인터넷으로 연결 가능한 EC2 생성 불가능